## get elevation from shp point and one dsm image

In [ ]:
import os
import rasterio
import geopandas as gpd
import pandas as pd

# Function to extract elevation from DSM TIFF file
def extract_elevation(point_shapefile, dsm_tiff):
    # Open point shapefile to get point IDs
    gdf = gpd.read_file(point_shapefile)
    point_ids = gdf["point_id"]

    # Open DSM TIFF file
    with rasterio.open(dsm_tiff) as src:
        # Extract elevation values for each point
        elevations = []
        for index, point in gdf.iterrows():
            x = point.geometry.x
            y = point.geometry.y
            # Sample elevation from DSM at point coordinates
            for val in src.sample([(x, y)]):
                elevations.append(val[0])

    # Create a DataFrame with point IDs and elevations
    df = pd.DataFrame({"point_id": point_ids, "Elevation": elevations})

    return df

# Function to write data to Excel
def write_to_excel(dataframe, excel_filename):
    dataframe.to_excel(excel_filename, index=False)

# Example usage
os.chdir(r'D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_06052023')
point_shapefile = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_06052023\inter_point_605.shp"
dsm_tiff = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_06052023\auora_f1_altum_pt_06052023_dsm.tif"
excel_filename = "elevation_data_auf1_605.xlsx"

elevation_data = extract_elevation(point_shapefile, dsm_tiff)
write_to_excel(elevation_data, excel_filename)


## chip for height images

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import os

# Function to extract chips from the raster based on the polygon geometry
def extract_chips(shapefile, raster_tif, output_dir):
    # Load the shapefile
    gdf = gpd.read_file(shapefile)

    # Load the raster TIF image
    with rasterio.open(raster_tif) as src:
        for index, row in gdf.iterrows():
            # Get the geometry of the polygon
            geom = row.geometry

            # Create a GeoJSON-like object to use as the mask
            geojson_geom = mapping(geom)

            # Extract the chip from the raster using the mask
            out_image, out_transform = mask(src, [geojson_geom], crop=True)

            # Get the metadata from the original raster
            out_meta = src.meta.copy()

            # Update the metadata for the chip
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            # Save the chip as a new raster file
            output_filename = os.path.join(output_dir, f"{row['Plot_ID']}.tif") # output_filename = os.path.join(output_dir, f"chip_{row['plot_id']}.tif")
            with rasterio.open(output_filename, "w", **out_meta) as dest:
                dest.write(out_image)
            

# Example usage
shapefile_path = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_06142023\auf1_06142023.shp"
raster_tif_path = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_06142023\chm_614.tif"
output_directory = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_614_height_chip"

extract_chips(shapefile_path, raster_tif_path, output_directory)


## extract height from chip images

In [ ]:
import os
import rasterio
import pandas as pd
import numpy as np

# Function to compute statistics from a single TIFF image
def compute_statistics(image_path):
    with rasterio.open(image_path) as src:
        data = src.read(1)
        mask = data != src.nodata
        height_values = data[mask]

        statistics = {
            'Hmean': np.mean(height_values),
            'Hmedian': np.median(height_values),
            'Hmin': np.min(height_values),
            'Hmax': np.max(height_values),
            'Hstd': np.std(height_values),
            'Hp90': np.percentile(height_values, 90),
            'Hp93': np.percentile(height_values, 93),
            'Hp95': np.percentile(height_values, 95),
            'Hp98': np.percentile(height_values, 98),
            'Hp99': np.percentile(height_values, 99)
        }
        return statistics

# Path to the folder containing the TIFF images
os.chdir(r'D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023')
images_folder = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_517_height_chip_2"

# List to store computed statistics for each image
statistics_list = []

# Iterate through each TIFF image in the folder
for filename in os.listdir(images_folder):
    if filename.endswith('.tif'):
        image_path = os.path.join(images_folder, filename)
        image_statistics = compute_statistics(image_path)
        image_statistics['Image'] = filename.split('.')[0]  # Extract image number from filename
        statistics_list.append(image_statistics)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(statistics_list)

# Reorder columns with 'Image' column as the first column
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]  # Moving 'Image' column to the beginning
df = df[cols]

# Path to save the Excel file
excel_file_path = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\height_auf1_wheat_lai_05172023.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print("Statistics saved to:", excel_file_path)


## extract height from chip images (error during the computation of statistics)

In [ ]:
import os
import rasterio
import pandas as pd
import numpy as np

# Function to compute statistics from a single TIFF image
def compute_statistics(image_path):
    try:
        with rasterio.open(image_path) as src:
            data = src.read(1)
            mask = data != src.nodata
            height_values = data[mask]

            if height_values.size == 0:
                raise ValueError("No valid data in the image")

            statistics = {
                'Hmean': np.mean(height_values),
                'Hmedian': np.median(height_values),
                'Hmin': np.min(height_values),
                'Hmax': np.max(height_values),
                'Hstd': np.std(height_values),
                'Hp90': np.percentile(height_values, 90),
                'Hp93': np.percentile(height_values, 93),
                'Hp95': np.percentile(height_values, 95),
                'Hp98': np.percentile(height_values, 98),
                'Hp99': np.percentile(height_values, 99)
            }
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        statistics = {
            'Hmean': '',
            'Hmedian': '',
            'Hmin': '',
            'Hmax': '',
            'Hstd': '',
            'Hp90': '',
            'Hp93': '',
            'Hp95': '',
            'Hp98': '',
            'Hp99': ''
        }

    return statistics

# Path to the folder containing the TIFF images
os.chdir(r'D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023')
images_folder = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\auf1_510_height_chip"

# List to store computed statistics for each image
statistics_list = []

# Iterate through each TIFF image in the folder
for filename in os.listdir(images_folder):
    if filename.endswith('.tif'):
        image_path = os.path.join(images_folder, filename)
        image_statistics = compute_statistics(image_path)
        image_statistics['Image'] = filename.split('.')[0]  # Extract image number from filename
        statistics_list.append(image_statistics)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(statistics_list)

# Reorder columns with 'Image' column as the first column
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]  # Moving 'Image' column to the beginning
df = df[cols]

# Path to save the Excel file
excel_file_path = r"D:\DATA_3RD_PAPER\WHEAT_AURORA_F1_NRPN_SRPN_ELITE_AYT_2023\height_auf1_wheat_lai_05102023.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print("Statistics saved to:", excel_file_path)


## get elevation from shp point and multiple dsm images from folder

In [ ]:
import os
import rasterio
import geopandas as gpd
import pandas as pd

# Function to extract elevation from DSM TIFF file
def extract_elevation(point_shapefile, dsm_tiff):
    # Open point shapefile to get point IDs
    gdf = gpd.read_file(point_shapefile)
    point_ids = gdf["point_id"]

    # Open DSM TIFF file
    with rasterio.open(dsm_tiff) as src:
        # Extract elevation values for each point
        elevations = []
        for index, point in gdf.iterrows():
            x = point.geometry.x
            y = point.geometry.y
            # Sample elevation from DSM at point coordinates
            for val in src.sample([(x, y)]):
                elevations.append(val[0])

    # Create a DataFrame with point IDs and elevations
    df = pd.DataFrame({"point_id": point_ids, "Elevation": elevations})

    return df

# Function to write data to Excel
def write_to_excel(dataframe, excel_filename):
    dataframe.to_excel(excel_filename, index=False)

# Main function to process all TIFF files in a folder
def process_tiff_files_in_folder(folder_path, point_shapefile):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".tif"):
            tiff_file_path = os.path.join(folder_path, file_name)
            excel_filename = os.path.splitext(file_name)[0] + "_elevation_data.xlsx"
            elevation_data = extract_elevation(point_shapefile, tiff_file_path)
            write_to_excel(elevation_data, excel_filename)

# Example usage
os.chdir(r'C:\maruf_files\prac')
folder_path = r"C:\maruf_files\prac\tiff_folder"
point_shapefile = r"C:\maruf_files\prac\interpol_point_project_706.shp"

process_tiff_files_in_folder(folder_path, point_shapefile)
